In [1]:
import numpy as np

In [2]:
with open("Embedding/movieEmbedding3.txt", encoding="utf-8") as embFile:
    total_file = embFile.read().split("\n")
with open("ID_Title.tsv") as embFile:
    title_file = embFile.read().split("\n")
title_list = []
for line in title_file[1:-1]:
    title_list.append(line.split("\t")[1])

In [3]:
def parse_line(line:str):
    for i in range(len(line)-1,-1,-1):
        if line[i] == "[":
            index = i
            break
    title = line[:i-1]
    emb = line[i+1:]
    return title, emb

In [4]:
def parse_emb(emb:str):
    emb_out = []
    start = 0
    end = 0
    flag = True
    sFlag = True
    for i in range(len(emb)):
        if emb[i] != " " and sFlag == True:
            start = i
            sFlag = False
            flag = False
        if emb[i] == " " and flag == False:
            end = i
            flag = True
        if emb[i] != " " and flag == True:
            emb_out.append(float(emb[start:end]))
            start = i
            flag = False
    if flag == True:
        emb_out.append(float(emb[start:end]))
    if end < len(emb):
        try:
            emb_out.append(float(emb[end:]))
        except:
            pass
    return emb_out

In [5]:
lol = 0
total_emb = []
emb_title = []
for i in range(len(total_file)):
    line = total_file[i]
    if line.find("[") != -1:
        title, emb = parse_line(line)
        emb_title.append(title)
        for j in range(i+1, i+13):
            if total_file[j].find("]") != -1:
                index = j
                lol += 1
                emb += total_file[j][:-1]
                break
            else:
                emb += "  " + total_file[j]
        emb_movie = parse_emb(emb)
        total_emb.append(emb_movie)
total_emb = np.array(total_emb)
print(lol)
print(len(total_emb))

34886
34886


In [8]:
ID_emb = np.zeros([len(title_list),max_emb_l])
lol = 0 
Overlap_list = []
for i in range(len(total_emb)):
    title = emb_title[i]
    try:
        kth = title_list.index(title)
        ID_emb[kth,:] = total_emb[i]
        lol += 1
        Overlap_list.append(kth)
    except:
        pass
print(lol)
Overlap_list = list(set(Overlap_list))
print(len(Overlap_list))

10370
9158


In [9]:
rating = np.load("npRatings.npy")
rating_ID = []
for i in range(len(rating)):
    if rating[i] != -1:
        rating_ID.append(i)
print(len(rating_ID))

19199


In [12]:
def Ov(list1, list2):
    return list(set(list1) & set(list2))
print(len(Ov(rating_ID, Overlap_list)))
OvListTTT =Ov(rating_ID, Overlap_list)
np.save("ovIDs", OvListTTT)

8510


In [13]:
attr = np.load("npHot.npy")
attrWEmb = []
for ID in Ov(rating_ID, Overlap_list):
    attrWEmb.append(np.concatenate([ID_emb[ID,:], attr[ID,:], rating[ID].reshape([1])]))
attrWEmb = np.array(attrWEmb)
# np.save("npAttrEmbOv.npy", attrWEmb)

In [14]:
sss = np.random.choice(list(range(len(total_emb))), 10)
max_emb_l = 0
min_emb_l = 1000
for item in total_emb:
    emb_l = len(item)
    if emb_l > max_emb_l:
        max_emb_l = emb_l
    if emb_l < min_emb_l:
        min_emb_l = emb_l
print(max_emb_l)
print(min_emb_l)

50
50


In [15]:
OvList = Ov(rating_ID, Overlap_list)
with open("title_DvI.txt") as f:
    DvI_file = f.read().split("\n")
DvI_ID = []
for ID in OvList:
    if title_list[ID] in DvI_file:
        DvI_ID.append(ID)
DmI_ID = list(set(OvList) - set(DvI_ID))
print(len(OvList))
print(len(DvI_ID))
print(len(DmI_ID))

8510
2782
5728


In [16]:
new_index_DvI = []
for i in DvI_ID:
    new_index_DvI.append(OvList.index(i))
new_index_DmI = []
for i in DmI_ID:
    new_index_DmI.append(OvList.index(i))
np.save("IMDBIntersectDouban", new_index_DvI)
np.save("IMDBDifferenceDouban", new_index_DmI)
# np.save("npIMDBvsDoubanSetOp.npy", index_dict)

In [40]:
lol = 0
for i in range(len(total_file)):
    line = total_file[i]
    if line == "":
        try:
            t = title_file[lol]
            t_n = total_file[i+1].split("[")[0][:-1]
        
            if t != t_n and total_file[i+1][0] !="[":
                print(t, t_n)
                print(lol)
                break
            else:
                lol += 1
        except:
            print(lol)
print(lol)

19964


In [5]:
with open("Genre_ID.tsv", "r") as f:
    genreList = []
    for line in f.read().split("\n")[:-1]:
        genreList.append(line.split("\t")[1])
with open("Language_ID.tsv", "r") as f:
    languageList = []
    for line in f.read().split("\n")[:-1]:
        languageList.append(line.split("\t")[1])
with open("Region_ID.tsv", "r") as f:
    regionList = []
    for line in f.read().split("\n")[:-1]:
        regionList.append(line.split("\t")[1])

In [6]:
class Movie():
    def __init__(self, attributes, genreList, languageList, regionList):
        self.ID = None
        self.title = attributes[2]
        self.isAdult = int(attributes[3])
        self.startYear = self.get_num(attributes[4])
        self.endYear = self.get_num(attributes[5])
        self.length = self.get_num(attributes[6])
        self.genres = self.get_cat(attributes[7], genreList)
        self.region = self.get_cat(attributes[8], regionList)[0]
        self.language = self.get_cat(attributes[9], languageList)[0]
        self.rating = self.get_num(attributes[10])
    
    def get_cat(self, attribute:str, aList:list):
        myList = []
        for a in attribute.replace("\"","").split(","):
            index = aList.index(a) - 1
            myList.append(index)
        return myList
    
    def get_num(self, attribute:str):
        try:
            attribute = float(attribute)
        except:
            attribute = -1.
        return attribute

In [7]:
all_movie = []
i = 0
for line in total_file[1:-1]:
    m = Movie(line.split("\t"), genreList, languageList, regionList)
    if m.rating != "\\N":
        m.ID = i
        i +=1
        all_movie.append(m)

In [8]:
with open("ID_Title.tsv", "w+") as f:
    f.write("ID\tTitle\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + m.title + "\n")

In [9]:
with open("ID_Rating.tsv", "w+") as f:
    f.write("ID\tRating\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + str(m.rating) + "\n")

In [11]:
with open("ID_Attributes.tsv", "w+") as f:
    f.write("ID\tisAdult\tstartYear\tendYear\tlength\tgenres\tregion\tlanguage\n")
    for m in all_movie:
        f.write(str(m.ID) + "\t" + str(m.isAdult) + "\t" + str(m.startYear) + "\t" + str(m.endYear) + "\t" + str(m.length) + "\t" + str(m.genres).replace("[","").replace("]","")  + "\t" + str(m.region)  + "\t" + str(m.language) +"\n")

In [21]:
npAttributes = np.zeros([len(all_movie),10])
def expand(mIn, size):
    small = len(mIn)
    assert small <= size
    r = [-1 for _ in range(size)]
    r[:small] = mIn 
    return r

for i, m in enumerate(all_movie):
    npAttributes[i,:5] = [m.ID, m.isAdult,m.startYear,m.endYear,m.length]
    npAttributes[i,5:8] = expand(m.genres,3)
    npAttributes[i,8:] = [m.region, m.language]
np.save("npAttributes",npAttributes)

In [22]:
ratings = []
for i, m in enumerate(all_movie):
    ratings.append(m.rating)
np.save("npRatings", ratings)